In [0]:
from pyspark.sql.functions import col, lit, when
from delta.tables import DeltaTable
from datetime import datetime, date
import json

In [0]:
try:

    v_init_json = dbutils.notebook.run(
        "/Workspace/EDW/utility/nb_init_run",
        60,
        {"notebook_name": "nb_fact_sale_transactions"}
    )

except Exception as e:
    print("Notebook 'nb_init_run' failed:")
    print(e)
    raise

In [0]:
print(v_init_json)

v_init_params = json.loads(v_init_json)

v_run_id = v_init_params["run_id"]
v_valid_from_dttm = datetime.fromisoformat(v_init_params["valid_from_dttm"]).date()
v_max_valid_to_dttm = datetime.fromisoformat(v_init_params["max_valid_to_dttm"]).date()
v_process_start_time = datetime.fromisoformat(v_init_params["process_start_time"])

print(v_run_id)
print(v_valid_from_dttm)
print(v_max_valid_to_dttm)
print(v_process_start_time)

In [0]:
spark.conf.set("fs.azure.account.key.stvptest001.dfs.core.windows.net", "QBM5+zw4iTqnSybwHeAZbFW38GQZXY9ch8EzO4IoIhwnxpoGV5mKIuKpxb1OFfhyftURUuMzZKGt+ASt8tCIhQ==")

In [0]:
df_br_sales = spark.read.table("bronze.sales")

In [0]:

df_cleaned_sales = df_br_sales.selectExpr(
        "TransactionID AS transaction_id",
        "Date AS sale_date",
        "ProductID AS product_id",
        "CAST(Quantity AS INT) AS quantity",
        "CAST(UnitPrice AS DECIMAL(18,2)) AS unit_price",
        "CAST(TotalPrice AS DECIMAL(18,2)) AS total_price",
        "CustomerID AS customer_id",
        "PaymentMethod AS payment_method",
        "Region AS region_name"
)

In [0]:
display(df_cleaned_sales)

In [0]:
df_dim_products = spark.read.table("silver.dim_products")

df_dim_products = df_dim_products.filter(col("ActiveInd") == lit("Y"))

df_silver_sales = (
    df_cleaned_sales.alias("src").join(
        df_dim_products.alias("dp"), 
        col("src.product_id") == col("dp.ProductID"),
        "left_outer"
    )
    .select(
        "src.*", 
        when(col("dp.ProductSK").isNull(), lit(-1))
        .otherwise(col("dp.ProductSK"))
        .alias("product_sk")
    )
)


In [0]:
display(df_silver_sales)

In [0]:
df_silver_sales.columns